In [1]:
import os
import inspect
print(os.getcwd())
plot_folder = '3d_arm_dyn_plots'

/Users/danbiderman/Dropbox/Columbia/1.Dan/John/aesmc/test


In [2]:
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)

In [3]:
parent_dir

'/Users/danbiderman/Dropbox/Columbia/1.Dan/John/aesmc'

In [4]:
# change dir to import from aesmc
os.chdir(parent_dir)
import aesmc.statistics as statistics
import aesmc.inference as inference
import aesmc.train as train
import aesmc.losses as losses
import aesmc.state as state
import aesmc.math as math
import aesmc.smoothing as smoothing

In [5]:
# change dir to import from aesmc/test
os.chdir(current_dir)
from arm_models import arm_3d_dyn_model # the model script -- distribution objects are defined there
from arm_models import arm_utils # make video, plot post, params to coords
from arm_models import utils_summary
#from arm_models import fw_sim_planar_arm_dyn

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

In [7]:
# define global model params
dt = 0.03 # time interval between observations
g = 1.0 # gravity 
dim_latents = 12 # always 6 in the full planar arm model.
transition_force_scale = 1.0 # 20.0 was best; tried also 10, 30, 50
transition_aux_scale = ((dt**2)*transition_force_scale**2) / (10.0) #* 2
initial_loc = 0.0 # natural choice.
initial_scale = 0.5 # in future could be different per dimension.
emission_var_diag = 0.02 # was 0.1 before. this varies with the application. 
print('torque effective variance: %.5f' %((dt**2)*transition_force_scale**2))
print('auxilliary variance: %.5f' % transition_aux_scale)

torque effective variance: 0.00090
auxilliary variance: 0.00009


In [8]:
L1_test = 1.0
L2_test = 1.0
M1_test = 0.3
M2_test = 0.5
t1 = 1.2
t2 = 1.4
t3 = 1.6
t4 = 1.1
dt1 = 1.5
dt2 = 2.2
dt3 = 3.1
dt4 = 0.4

In [9]:
# put params into dicts
param_dict = {}
param_dict["init_mu"] = initial_loc * np.ones(dim_latents)
param_dict["init_cov"] = np.eye(dim_latents) * initial_scale

inits_dict = {}
inits_dict["L1"] = L1_test
inits_dict["L2"] = L2_test
inits_dict["M1"] = M1_test 
inits_dict["M2"] = M2_test 

In [11]:
# initialize arm model, used by transition and proposal.
# below we include_gravity_fictitious = True and transform_torques
arm_model_instance = arm_3d_dyn_model.Arm_3D_Dyn(
    dt=dt,
    inits_dict=inits_dict, # make sure
    g=g,
    include_gravity_fictitious=False,
    transform_torques=False,
    learn_static=False,
    restrict_to_plane=True)

In [12]:
D_torch = arm_model_instance.D(t2=torch.tensor(np.ones(1) * t2),
                     t3=torch.tensor(np.ones(1) * t3),
                     t4=torch.tensor(np.ones(1) * t4))

In [26]:
import dill
parent_parent_dir = os.path.dirname(parent_dir)

In [48]:
D_func_lambdify = dill.load(
    open(
        os.path.join(parent_parent_dir, 'arm_notebooks', 'lambdify_funcs',
                     'D_func'), "rb"))

In [49]:
out_D = D_func_lambdify(L1=L1_test,
           L2=L2_test,
           M1=M1_test,
           M2=M2_test,
           t1=t1,
           t2=t2,
           t3=t3,
           t4=t4)
out_D_arr = np.vstack([out_D[0], out_D[1], out_D[2], out_D[3]])

In [50]:
tolerance = 0.0001
((torch.tensor(out_D_arr) - D_torch) < tolerance).numpy().all()

True

In [51]:
h_func_lambdify = dill.load(
    open(
        os.path.join(parent_parent_dir, 'arm_notebooks', 'lambdify_funcs',
                     'h_func'), "rb"))

In [52]:
h_func_lambdify

<function _lambdifygenerated(L1, L2, M1, M2, t1, t2, t3, t4, dt1, dt2, dt3, dt4)>

In [54]:
out_h = h_func_lambdify(L1=L1_test,
                        L2=L2_test,
                        M1=M1_test,
                        M2=M2_test,
                        t1 = t1,
                        t2 = t2,
                        t3 = t3,
                        t4 = t4,
                        dt1 = dt1,
                        dt2 = dt2,
                        dt3 = dt3,
                        dt4 = dt4)

In [61]:
h_torch = arm_model_instance.h(t2=torch.tensor(np.ones(1) * t2),
                     t3=torch.tensor(np.ones(1) * t3),
                     t4=torch.tensor(np.ones(1) * t4),
                    dt1 = torch.tensor(np.ones(1) * dt1),
                    dt2 = torch.tensor(np.ones(1) * dt2),
                    dt3 = torch.tensor(np.ones(1) * dt3), 
                    dt4 = torch.tensor(np.ones(1) * dt4))

In [71]:
tolerance = 0.0001
((torch.tensor(out_h) - h_torch.reshape(len(out_h))) < tolerance).numpy().all()

True

In [72]:
c_func_lambdify = dill.load(
    open(
        os.path.join(parent_parent_dir, 'arm_notebooks', 'lambdify_funcs',
                     'c_func'), "rb"))

In [75]:
out_c = c_func_lambdify(g=g,
                L1=L1_test,
                L2=L2_test,
                M1=M1_test,
                M2=M2_test,
                t1=t1,
                t2=t2,
                t3=t3,
                t4=t4)

In [78]:
c_torch = arm_model_instance.c(t1=torch.tensor(np.ones(1) * t1),
                     t2=torch.tensor(np.ones(1) * t2),
                     t3=torch.tensor(np.ones(1) * t3),
                     t4 = torch.tensor(np.ones(1) *t4))

In [79]:
tolerance = 0.0001
((torch.tensor(out_c) - c_torch.reshape(len(out_c))) < tolerance).numpy().all()

True

In [80]:
#TODO - test the forward kinematics.

In [101]:
np.random.seed(100)
torque_vec_test = torch.tensor(np.random.normal(size = 4)).view(1,4,1)
print(torque_vec_test)
torque_vec_test_numpy = torque_vec_test.numpy().reshape(4,1)
print(torque_vec_test_numpy)

tensor([[[-1.7498],
         [ 0.3427],
         [ 1.1530],
         [-0.2524]]], dtype=torch.float64)
[[-1.74976547]
 [ 0.3426804 ]
 [ 1.1530358 ]
 [-0.25243604]]


In [94]:
print(torque_vec_test.shape)
print(D_torch.shape)
print(h_torch.shape)
print(c_torch.shape)

torch.Size([1, 4, 1])
torch.Size([1, 4, 4])
torch.Size([1, 4, 1])
torch.Size([1, 4, 1])


In [111]:
# test newton's second law
accel_torch = arm_model_instance.Newton_2nd(torque_vec_tens = torque_vec_test, 
                                      D_mat_tens = D_torch, 
                                      h_vec_tens = h_torch, 
                                      c_vec_tens = c_torch)
accel_torch

tensor([[[-177.0820],
         [  14.4122],
         [  30.4787],
         [ 115.1055]]], dtype=torch.float64)

In [104]:
def make_column_vec(list_arg):
    return np.asarray(list_arg).reshape(len(list_arg), 1)
#make_column_vec(c_out)

In [109]:
h_on = 1
c_on = 1
D_inv = np.linalg.inv(out_D)  # compute using previous theta 2,3 and 4
brackets = torque_vec_test_numpy - h_on * make_column_vec(out_h) \
- c_on*make_column_vec(out_c)
accel_numpy = np.dot(D_inv, brackets).squeeze()

In [114]:
((accel_numpy-accel_torch.numpy().reshape(4))<tolerance).all()

True

In [112]:
accel_torch.shape

torch.Size([1, 4, 1])

In [129]:
mean_fully_expanded = torch.tensor(np.random.normal(size=(32, 100, 12)))

In [130]:
mean_fully_expanded[:,:,[1,3,5,7,9,11]] = torch.zeros(1, 
                                                        dtype = torch.double)

In [131]:
# simulate forward

tensor([[[ 0.3832,  0.0000,  0.5264,  ...,  0.0000, -0.1202,  0.0000],
         [ 0.0925,  0.0000, -0.3966,  ...,  0.0000,  0.3664,  0.0000],
         [-0.7108,  0.0000, -0.6515,  ...,  0.0000, -0.5444,  0.0000],
         ...,
         [ 1.1376,  0.0000, -0.0855,  ...,  0.0000, -0.5793,  0.0000],
         [ 1.1626,  0.0000, -1.6926,  ...,  0.0000, -0.2253,  0.0000],
         [-0.0437,  0.0000,  0.4778,  ...,  0.0000,  0.0829,  0.0000]],

        [[ 0.0812,  0.0000,  0.0664,  ...,  0.0000, -2.1563,  0.0000],
         [ 0.4721,  0.0000, -0.3681,  ...,  0.0000, -1.1282,  0.0000],
         [ 1.0165,  0.0000, -0.5722,  ...,  0.0000, -0.3996,  0.0000],
         ...,
         [-1.2281,  0.0000, -0.1163,  ...,  0.0000,  0.6322,  0.0000],
         [ 0.1245,  0.0000, -1.0935,  ...,  0.0000,  1.1863,  0.0000],
         [-0.4456,  0.0000, -1.1137,  ...,  0.0000, -0.2440,  0.0000]],

        [[-0.4961,  0.0000, -0.4521,  ...,  0.0000,  1.5372,  0.0000],
         [-0.4989,  0.0000,  0.7467,  ...,  0